# %%bash

pip install --upgrade pip
pip install farm-haystack


In [1]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


In [2]:
doc_dir = r'D:\grduation_project\datasets'

In [3]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Updating BM25 representation...: 100%|███████████████████████████████████████████████| 4/4 [00:00<00:00, 255.96 docs/s]


{'documents': [<Document: {'content': 'The Qaitbay Citadel in Alexandria is considered one of the most important defensive strongholds, not only in Egypt, but also along the Mediterranean Sea coast. It formulated an important part of the fortification system of Alexandria in the 15th century AD.[4][5][6]\n\nLighthouse of Alexandria\nThe Citadel is situated at the entrance of the eastern harbour on the eastern point of Pharos Island. It was erected on the site of the former Lighthouse of Alexandria, which was one of the Seven Wonders of the Ancient World. Many structural modifications were made to the lighthouse after the Arab conquest of Egypt, but the lighthouse continued to function for many centuries. Restoration began in the period of Ahmed Ibn Tulun (about 880 AD). During the 11th century an earthquake occurred, causing damage to the octagonal part. The bottom survived, but it could only serve as a watchtower, and a small mosque was built on the top. Two very destructive earthquak

In [4]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [5]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


D:\Huggingface\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:  46%|####6     | 231M/496M [00:00<?, ?B/s]

D:\Huggingface\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mega Store\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [6]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


In [50]:
q=input("Enter Qustion")
prediction = pipe.run(
    query=q, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)


Enter Qustion what is history of qaitbay


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.92s/ Batches]


In [51]:
prediction['answers'][0].answer

'The Qaitbay Citadel in Alexandria is considered one of the most important defensive strongholds, not only in Egypt, but also along the Mediterranean Sea coast. It formulated an important part of the fortification system of Alexandria in the 15th century AD.[4][5][6]\n\nLighthouse of Alexandria\nThe Citadel is situated at the entrance of the eastern harbour on the eastern point of Pharos Island. It was erected on the site of the former Lighthouse of Alexandria, which was one of the Seven Wonders of the Ancient World. Many structural modifications were made to the lighthouse after the Arab conquest of Egypt, but the lighthouse continued to function for many centuries'

In [ ]:
from pprint import pprint

pprint(prediction)


In [ ]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`


In [1]:
import joblib

In [54]:
joblib.dump(pipe,'D:\grduation_project\chat_bot.pkl')

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Mega Store\AppData\Local\Temp\ipykernel_7920\2410186498.py:1: SyntaxWarning: invalid escape sequence '\g'
  joblib.dump(pipe,'D:\grduation_project\chat_bot.pkl')


['D:\\grduation_project\\chat_bot.pkl']

In [2]:
pipe=joblib.load(r'D:\\grduation_project\\chat_bot.pkl')

In [13]:
q=input("Enter Qustion")
prediction = pipe.run(
    query=q, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)


Enter Qustion who is build quitbay citadel


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.04s/ Batches]


In [14]:
prediction['answers'][0].answer

'Qagmas Al-Eshaqy'